<a href="https://colab.research.google.com/github/chrober24/A.UD_289.84_seminar/blob/main/3D_Photo_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[CVPR 2020] 3D Photography using Context-aware Layered Depth Inpainting**

[project website](https://shihmengli.github.io/3D-Photo-Inpainting/)

# Setup


In [ ]:
!pip3 install vispy==0.6.4 moviepy==1.0.2 networkx==2.3 transforms3d==0.3.1

- **Download script and pretrained model**


In [ ]:
%cd /content/
!git clone https://github.com/vt-vl-lab/3d-photo-inpainting.git
%cd 3d-photo-inpainting
!sh download.sh

- **Switch off off-screen rendering**

In [6]:
!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

# Upload Images
**Please upload `.jpg` or `.png` files to `/content/3d-photo-inpainting-master/image/`**

You can run this step multiple times to upload multiple `.jpg/.png` files.

In [ ]:
#@title Upload Images

%cd image
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
%cd ..

# Write new Argument File
edit the arguments in this file to control settings for the inpainting

Important Settings will have a green comment next to them

A detailed explanation of all settings can be found here: ```/content/3d-photo-inpainting/DOCUMENTATION.md```

important settings: 


*   line 7: fps
*   line 8: num_frames
*   line 14: specific-- specify single image to process. otherwise, all images in the folder will be processed
*   line 15: longer_side_len-- dimension of output (higher than 960 may run out of memory)
*   line 25: img_format: change to .png if using png format



In [ ]:
%%writefile argument.yml
depth_edge_model_ckpt: checkpoints/edge-model.pth
depth_feat_model_ckpt: checkpoints/depth-model.pth
rgb_feat_model_ckpt: checkpoints/color-model.pth
MiDaS_model_ckpt: MiDaS/model.pt
use_boostmonodepth: True
fps: 40
num_frames: 240
x_shift_range: [0.00, 0.00, -0.015, -0.015]
y_shift_range: [0.00, 0.00, -0.015, -0.00]
z_shift_range: [-0.05, -0.05, -0.05, -0.05]
traj_types: ['double-straight-line', 'double-straight-line', 'circle', 'circle']
video_postfix: ['dolly-zoom-in', 'zoom-in', 'circle', 'swing']
specific: ''
longer_side_len: 960
src_folder: image
depth_folder: depth
mesh_folder: mesh
video_folder: video
load_ply: False
save_ply: True
inference_video: True
gpu_ids: 0
offscreen_rendering: False
img_format: '.jpg'
depth_format: '.npy'
require_midas: True
depth_threshold: 0.04
ext_edge_threshold: 0.002
sparse_iter: 5
filter_size: [7, 7, 5, 5, 5]
sigma_s: 4.0
sigma_r: 0.5
redundant_number: 12
background_thickness: 70
context_thickness: 140
background_thickness_2: 70
context_thickness_2: 70
discount_factor: 1.00
log_depth: True
largest_size: 512
depth_edge_dilate: 10
depth_edge_dilate_2: 5
extrapolate_border: True
extrapolation_thickness: 60
repeat_inpaint_edge: True
crop_border: [0.03, 0.03, 0.05, 0.03]
anti_flickering: True

# Execute the 3D Photo Inpainting
  - Note: The 3D photo generation process usually takes about 2-3 minutes or more depending on the available computing resources.

In [ ]:
!python main.py --config argument.yml

- **The results are stored in the following directories**
  - Corresponding depth map estimated by [MiDaS](https://github.com/intel-isl/MiDaS.git) 
      - E.g. ```/content/3d-photo-inpainting/depth/moon.npy```
  - Inpainted 3D mesh
      - E.g. ```/content/3d-photo-inpainting/mesh/moon.ply```
  - Rendered videos with zoom-in motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_zoom-in.mp4```
  - Rendered videos with swing motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_swing.mp4```
  - Rendered videos with circle motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_circle.mp4```
  
    ![](https://drive.google.com/uc?id=1xBiZAeGfhsoAsG08N5UM8-f993p0G5HF)

In [ ]:
#@title Show Video

vid_path = '/content/3d-photo-inpainting/video/A1_zoom-in.mp4' #@param {type: 'string'}
W = 1024 #@param {type: 'number'}
H = 1024 #@param {type: 'number'}

def show_local_mp4_video(file_name, width=W, height=H):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video(vid_path, width= W, height= H)